## Load protobuf models
To build the protobuf files, I ran:
```
cd src;
protoc --python_out=generated --proto_path=generated schemas/nanoswap/message/*.proto schemas/nanoswap/enum/*.proto
```

In [34]:
import sys
from google.protobuf.json_format import MessageToJson
sys.path.append("./generated")

import nanoswap.enum.issuers_pb2 as issuers_pb2
import nanoswap.message.identity_pb2 as identity_pb2

## Create an example data record

In [50]:
identity = identity_pb2.Identity(
    id_field_type = issuers_pb2.Issuer.UNITED_STATES_AMERICA___FEDERAL___SOCIAL_SECURITY_NUMBER,
    id_field_content = "123-45-6789"
)

In [51]:
# Note: if the value of id_field_type is set to 1 (default value) it will not be included in the payload
MessageToJson(identity, including_default_value_fields=True)

'{\n  "idFieldContent": "123-45-6789",\n  "idFieldType": "UNITED_STATES_AMERICA___FEDERAL___SOCIAL_SECURITY_NUMBER"\n}'

## Serialize the example record to prepare to store it in ipfs

In [52]:
content = identity.SerializeToString()
content

b'\x12\x0b123-45-6789'

## Try to deserialize it

In [53]:
reader = identity_pb2.Identity()
reader.ParseFromString(content)
MessageToJson(reader, including_default_value_fields=True)

'{\n  "idFieldContent": "123-45-6789",\n  "idFieldType": "UNITED_STATES_AMERICA___FEDERAL___SOCIAL_SECURITY_NUMBER"\n}'

## Write the serialize message to ipfs

In [54]:
import uuid

# suppose this is the new credit identity we decided to issue for the user
credit_id = uuid.uuid4()
credit_id

UUID('6f1657bb-e624-4d8c-a3ed-07966229573b')

In [55]:
# uuid4 type conversion examples
print(credit_id.bytes)
print(str(credit_id))

b'o\x16W\xbb\xe6$M\x8c\xa3\xed\x07\x96b)W;'
6f1657bb-e624-4d8c-a3ed-07966229573b


In [56]:
filename = 'identity.' + str(credit_id)
filename

'identity.6f1657bb-e624-4d8c-a3ed-07966229573b'

## Write the file that we want to upload
Long term it would be better to use HTTP or somehow write it in memory, but this looked like the easiest method

In [57]:
filepath = "generated/tmp/" + filename
with open(filepath, "wb") as f:
    f.write(content)

## Upload the file to ipfs with the cli

In [58]:
import subprocess

In [104]:
subprocess.run(["ipfs", "files", "mkdir", "-p", "/data/"])


CompletedProcess(args=['ipfs', 'files', 'mkdir', '-p', '/data/'], returncode=0)

In [105]:
result = subprocess.run(["ipfs", "add", filepath, "--to-files", "/data/"], capture_output=True)
hash = result.stdout.strip()
hash

b'added QmUYz8RCMdcbaekmpGRNdDkVyfVghZwkoKr9d7KydFXQwP identity.6f1657bb-e624-4d8c-a3ed-07966229573b'

## Read the data back from ipfs

In [110]:
result = subprocess.run(["ipfs", "files", "ls", "/data/"], capture_output=True)
data = result.stdout.strip()
data

b'identity.6f1657bb-e624-4d8c-a3ed-07966229573b'

In [117]:
result = subprocess.run(["ipfs", "files", "read", "/data/" + filename], capture_output=True)
data = result.stdout.strip()
data

b'\x12\x0b123-45-6789'

In [122]:
reader2 = identity_pb2.Identity()
reader2.ParseFromString(data)
result = MessageToJson(reader2, including_default_value_fields=True)
result

'{\n  "idFieldContent": "123-45-6789",\n  "idFieldType": "UNITED_STATES_AMERICA___FEDERAL___SOCIAL_SECURITY_NUMBER"\n}'

In [129]:
reader2.id_field_type

0

In [130]:
reader2.id_field_content

'123-45-6789'